In [2]:
import json
import pandas as pd
from pandas.io.json import json_normalize
import requests
import folium
from sklearn.cluster import KMeans
from tabulate import tabulate

# Define your Foursquare API credentials and endpoint URLs
API_KEY = '32IpE2MmM_VTAjNlaD9riTY2zAjg0zzfX_lBiINwHAw'
VERSION = '20230912'  # Use the specific version date you prefer, e.g., 'YYYYMMDD'

# Set up the API request for apartment locations (modify the parameters)
location = '13.133521,77.567135'  # Specify the coordinates for your desired location
radius = 100000  # Adjust the radius as needed
query = 'apartment'  # Customize the query term

apartment_url = f'https://discover.search.hereapi.com/v1/discover?in=circle:{location};r={radius}&q={query}&apiKey={API_KEY}'
apartment_data = requests.get(apartment_url).json()
apartment_df = json_normalize(apartment_data['items'])

# Count the number of cafes, department stores, and gyms near each apartment
CafeList = []
DepList = []
GymList = []

for lat, lng in zip(apartment_df['position.lat'], apartment_df['position.lng']):
    radius = 1000  # Set the radius to 1000 meters
    latitude = lat
    longitude = lng

    for search_query, result_list in [('cafe', CafeList), ('gym', GymList), ('department-store', DepList)]:
        url = f'https://discover.search.hereapi.com/v1/discover?in=circle:{latitude},{longitude};r={radius}&q={search_query}&apiKey={API_KEY}'
        results = requests.get(url).json()
        venues = json_normalize(results['items'])
        result_list.append(venues['title'].count())

apartment_df['Cafes'] = CafeList
apartment_df['Department Stores'] = DepList
apartment_df['Gyms'] = GymList

# Display the data as a table
print(tabulate(apartment_df[['position.lat', 'position.lng', 'Cafes', 'Department Stores', 'Gyms']],
               headers='keys', tablefmt='github'))

# Run K-means clustering on the dataframe
kclusters = 3
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(apartment_df[['position.lat', 'position.lng']])
apartment_df['Cluster'] = kmeans.labels_.astype(str)

# Plot clustered locations on the map using Folium
map_location = folium.Map(location=[19.1334, 72.9133], zoom_start=12)
colors = ['green', 'orange', 'red']

for lat, lng, cluster in zip(apartment_df['position.lat'], apartment_df['position.lng'], apartment_df['Cluster']):
    folium.CircleMarker(
        [lat, lng],
        fill=True,
        fill_opacity=1,
        radius=5,
        color=colors[int(cluster)],
    ).add_to(map_location)

# Save the map
map_location.save("index.html")


<ipython-input-2-203d8ea12943>:20: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  apartment_df = json_normalize(apartment_data['items'])
<ipython-input-2-203d8ea12943>:35: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  venues = json_normalize(results['items'])
<ipython-input-2-203d8ea12943>:35: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  venues = json_normalize(results['items'])
<ipython-input-2-203d8ea12943>:35: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  venues = json_normalize(results['items'])
<ipython-input-2-203d8ea12943>:35: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  venues = json_normalize(results['items'])
<ipython-input-2-203d8ea12943>:35: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normal

|    |   position.lat |   position.lng |   Cafes |   Department Stores |   Gyms |
|----|----------------|----------------|---------|---------------------|--------|
|  0 |        13.0609 |        77.6328 |       1 |                   1 |      1 |
|  1 |        12.9952 |        77.6107 |       7 |                   3 |     12 |
|  2 |        12.9738 |        77.5484 |       6 |                   6 |     16 |
|  3 |        12.9728 |        77.5472 |       5 |                   6 |     17 |
|  4 |        12.9723 |        77.5515 |       7 |                   5 |     15 |
|  5 |        12.969  |        77.543  |       7 |                   8 |     20 |
|  6 |        12.9692 |        77.5408 |       9 |                   7 |     20 |
|  7 |        12.9686 |        77.5417 |       7 |                   7 |     20 |
|  8 |        12.9681 |        77.5419 |       7 |                   7 |     20 |
|  9 |        12.9671 |        77.5428 |       7 |                   6 |     20 |
| 10 |        12

In [3]:
apartment_df.to_csv('apartment_data.csv', index=False)


,title,id,language,ontologyId,resultType,access,distance,categories,address.label,address.countryCode,...,position.lng,contacts,references,openingHours,address.houseNumber,payment.methods,Cafes,Department Stores,Gyms,Cluster
0,Monarch,here:pds:place:356tdr4q-6d344f77bbac4d2b92f15c...,en,here:cm:ontology:apartment_rental_flat_rental,place,"[{'lat': 13.06086, 'lng': 77.63303}]",10758,"[{'id': '700-7200-0324', 'name': 'Apartment/Fl...","Monarch, Hegde Nagar Main Road, Railway Men La...",IND,...,77.63279,NaN,NaN,NaN,NaN,NaN,1,1,1,2
1,The Turning Point,here:pds:place:356tdr1y-e60ca61307424f8ab53e27...,en,here:cm:ontology:apartment_rental_flat_rental,place,"[{'lat': 12.99513, 'lng': 77.61076}]",16093,"[{'id': '700-7200-0324', 'name': 'Apartment/Fl...","The Turning Point, Saunders Road, Pulakeshi Na...",IND,...,77.61071,[{'phone': [{'value': '+919845031796'}]}],NaN,NaN,NaN,NaN,7,3,12,1
2,Mathrukrupa,here:pds:place:356jx7ps-7560eab8dea20f5c34ca52...,en-GB,here:cm:ontology:apartment_rental_flat_rental,place,"[{'lat': 12.97379, 'lng': 77.54841}]",17875,"[{'id': '900-9300-0221', 'name': 'Residential ...","Mathrukrupa, 3rd Cross Road, Vidyaranyanagar, ...",IND,...,77.54842,NaN,"[{'supplier': {'id': 'core'}, 'id': '106943647...",NaN,NaN,NaN,6,6,16,0
3,Matrushree Nilaya,here:pds:place:356jx7ps-b81ab8f7ccb20876122ac0...,en-GB,here:cm:ontology:apartment_rental_flat_rental,place,"[{'lat': 12.97284, 'lng': 77.54712}]",17996,"[{'id': '900-9300-0221', 'name': 'Residential ...","Matrushree Nilaya, 1st Cross Road, Manuvana, V...",IND,...,77.54715,NaN,"[{'supplier': {'id': 'core'}, 'id': '113017170...",NaN,NaN,NaN,5,6,17,0
4,Lakshmi Ramachandra Nilaya,here:pds:place:356jx7ps-dd6b49221d160040427bd0...,en-GB,here:cm:ontology:apartment_rental_flat_rental,place,"[{'lat': 12.97229, 'lng': 77.55147}]",18008,"[{'id': '900-9300-0221', 'name': 'Residential ...","Lakshmi Ramachandra Nilaya, 2nd Cross Road, Ma...",IND,...,77.55147,NaN,"[{'supplier': {'id': 'core'}, 'id': '113019769...",NaN,NaN,NaN,7,5,15,0


In [6]:
apartment_df.isnull().sum()
#Cafes, department and Gym are non null so we are good :-))

title                   0
id                      0
language                0
ontologyId              0
resultType              0
access                  0
distance                0
categories              0
address.label           0
address.countryCode     0
address.countryName     0
address.stateCode       0
address.state           0
address.county          0
address.city            0
address.district        0
address.subdistrict     9
address.street          2
address.postalCode      0
position.lat            0
position.lng            0
contacts               17
references              3
openingHours           18
address.houseNumber    19
payment.methods        19
Cafes                   0
Department Stores       0
Gyms                    0
Cluster                 0
dtype: int64